In [2]:
# Melakukan import dataset standard iris dan memisahkan data dan target
from sklearn import datasets
iris_dataset = datasets.load_iris()
X_iris = iris_dataset.data
Y_iris = iris_dataset.target
print(X_iris[0:5])
print(Y_iris[0:5])

[[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]
 [ 4.6  3.1  1.5  0.2]
 [ 5.   3.6  1.4  0.2]]
[0 0 0 0 0]


In [20]:
# Melakukan import dataset csv play tennis dan memisahkan data dan target
import pandas
ori_data_frame = pandas.read_csv('tennis.csv')

# axis = 1 artinya mau drop column
data_frame = ori_data_frame.drop('play', axis=1)
data_frame = data_frame.select_dtypes(include=[object])
# print(data_frame.columns)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_2 = data_frame.apply(le.fit_transform)
# print(df_2)

one_hot_encoder = preprocessing.OneHotEncoder()
one_hot_encoder.fit(df_2)
one_hot_labels = one_hot_encoder.transform(df_2).toarray()

X_tennis = one_hot_labels
Y_tennis = []

for row in ori_data_frame.index:
    Y_tennis.append(ori_data_frame['play'][row])

print(X_tennis[0:5])
print(Y_tennis[0:5])

[[ 0.  0.  1.  0.  1.  0.  1.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.]
 [ 1.  0.  0.  0.  1.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.  1.  1.  0.]
 [ 0.  1.  0.  1.  0.  0.  0.  1.]]
['no', 'no', 'yes', 'yes', 'yes']


In [21]:
# Melakukan pembelajaran full training dengan DecisionTree
from sklearn import tree
clf_tree = tree.DecisionTreeClassifier()
clf_tree = clf_tree.fit(X_iris, Y_iris)
clf_tree_2 = tree.DecisionTreeClassifier()
clf_tree_2 = clf_tree_2.fit(X_tennis, Y_tennis)
# Menampilkan hasil prediksi terhadap 5 instance awal
print(clf_tree.predict(X_iris[:5]))
print(clf_tree_2.predict(X_tennis[:5]))

[0 0 0 0 0]
['no' 'no' 'yes' 'no' 'no']


In [22]:
# Menampilkan model pembelajaran tree
data_feature_names = iris_dataset.feature_names
data_feature_names_2 = ['outlook', 'temp', 'humidity']
data_target_names = iris_dataset.target_names
data_target_names_2 = ['no', 'yes']
dot_model = tree.export_graphviz(clf_tree, feature_names = data_feature_names, class_names = data_target_names, out_file = None) 
dot_model_2 = tree.export_graphviz(clf_tree_2, class_names = data_target_names_2, out_file = None) 

# import graphviz
# import os
# os.environ["PATH"] += os.pathsep + 'D:/Graphviz2.38/bin/'
# tree_viz = graphviz.Source(dot_model) 
# tree_viz.render('tree_iris')
# tree_viz_2 = graphviz.Source(dot_model_2) 
# tree_viz_2.render('tree_tennis')

In [23]:
# Melakukan pembelajaran full training dengan MLP
from sklearn.neural_network import MLPClassifier
clf_mlp = MLPClassifier(solver='sgd', max_iter = 10000, alpha=1e-3, hidden_layer_sizes=(4,3))
clf_mlp.fit(X_iris, Y_iris)
clf_mlp_2 = MLPClassifier(solver='sgd', max_iter = 10000, alpha=1e-3, hidden_layer_sizes=(4,3))
clf_mlp_2.fit(X_tennis, Y_tennis)
# Menampilkan hasil prediksi terhadap 5 instance awal
print(clf_mlp.predict(X_iris[:5]))
print(clf_mlp_2.predict(X_tennis[:5]))

[2 2 2 2 2]
['no' 'no' 'yes' 'no' 'yes']


In [24]:
# Menampilkan model pembelajaran MLP untuk iris
for i in range(0, len(clf_mlp.coefs_)):
    print('Intercepts:')
    print(clf_mlp.intercepts_[i])
    print('Weights:')
    print(clf_mlp.coefs_[i])

Intercepts:
[ 0.41592897  0.55196247  0.83239129 -0.71110706]
Weights:
[[-0.1382848   0.68229231  0.24909409  0.32411711]
 [-0.518432    0.51740484 -0.61110884  0.41252773]
 [ 0.65464656  0.57507848  0.63393694  0.65820465]
 [-0.50000104 -0.48106133 -0.76205074  0.1092443 ]]
Intercepts:
[ 0.26277633  0.10340675  0.08663686]
Weights:
[[-0.81650394  0.5212896   0.79327185]
 [-0.38014156 -0.51016825  0.91078747]
 [ 0.19969301 -0.23068556 -0.50470873]
 [-0.01303121 -0.85660111  0.4508253 ]]
Intercepts:
[ 0.54715402  0.01619956  0.63516746]
Weights:
[[-0.60147417  0.75742537 -0.40320871]
 [ 0.61124912 -0.08296257  0.81689099]
 [ 0.29274183  0.35375793  0.33164474]]


In [25]:
# Menampilkan model pembelajaran MLP untuk tennis
for i in range(0, len(clf_mlp_2.coefs_)):
    print('Intercepts:')
    print(clf_mlp_2.intercepts_[i])
    print('Weights:')
    print(clf_mlp_2.coefs_[i])

Intercepts:
[-0.28598022  0.61307971 -0.5275994  -0.34328841]
Weights:
[[-0.53239992  1.02204545 -0.31435104  0.06451635]
 [-0.66902833 -0.62864548 -0.30967686  0.57419476]
 [ 0.24224535 -0.78062875  0.03141856  0.49717489]
 [ 0.7718902   0.05528242  0.61012204 -0.36995889]
 [ 0.1033249   0.01994762  0.11793707 -0.32557636]
 [ 0.54337935  0.05912507  0.48353137  0.85748106]
 [-0.40296526  0.14707378  0.01263783 -0.35082323]
 [-0.23042038  1.02211356  0.22715819  0.80059517]]
Intercepts:
[ 0.19515131 -0.84261841 -0.11632816]
Weights:
[[ 0.5406129  -0.70663917 -0.65671973]
 [ 1.62490651 -0.06819179 -0.28142461]
 [ 0.15620744  0.09603523 -0.68941385]
 [-0.25790016  1.38302679 -0.81220467]]
Intercepts:
[-0.93811645]
Weights:
[[ 1.0860389 ]
 [ 1.10476732]
 [ 0.72081298]]


In [26]:
# Melakukan import dan export model
from sklearn.externals import joblib
def save_model(model, filename):
    joblib.dump(model, filename)
    
def load_model(filename):
    clf = joblib.load(filename)
    return clf

In [28]:
# Meng-export full training MLP dan DTree
save_model(clf_tree, "full_training_Tree_Iris.model")
save_model(clf_mlp, "full_training_MLP_Iris.model")
save_model(clf_tree_2, "full_training_Tree_Tennis.model")
save_model(clf_mlp_2, "full_training_MLP_Tennis.model")

# Me-load full training MLP dan DTree
clf_mlp = load_model("full_training_MLP_Iris.model")
clf_tree = load_model("full_training_Tree_Iris.model")
clf_mlp_2 = load_model("full_training_MLP_Tennis.model")
clf_tree_2 = load_model("full_training_Tree_Tennis.model")

# Tes apakah berhasil load dengan melakukan prediksi
print(clf_mlp.predict(X_iris[:5]))
print(clf_tree.predict(X_iris[:5]))
print(clf_mlp_2.predict(X_tennis[:5]))
print(clf_tree_2.predict(X_tennis[:5]))

[2 2 2 2 2]
[0 0 0 0 0]
['no' 'no' 'yes' 'no' 'yes']
['no' 'no' 'yes' 'no' 'no']


In [28]:
#C ANN + Decision Tree Split test
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix

#Pisah data menjadi train dan test
irisTrainX, irisTestX, irisTrainY, irisTestY =  train_test_split(X_iris, Y_iris, test_size=0.1)

#Make Classifier
decisionTree = tree.DecisionTreeClassifier()
multiLayerANN = MLPClassifier(solver = "lbfgs")
decisionTree.fit(irisTrainX, irisTrainY)
multiLayerANN.fit(irisTrainX, irisTrainY)

treePredict = decisionTree.predict(irisTestX)
annPredict = multiLayerANN.predict(irisTestX)

print(treePredict)
treeAccuracy = accuracy_score(irisTestY, treePredict)
treeMatrix = confusion_matrix(irisTestY, treePredict)
print()
print("Accuracy : ") 
print(treeAccuracy)
print()
print("Confusion Matrix :")
print(treeMatrix)
print()

print(annPredict)
annAccuracy = accuracy_score(irisTestY, annPredict)
annMatrix = confusion_matrix(irisTestY, annPredict)
print()
print("Accuracy : ") 
print(annAccuracy)
print()
print("Confusion Matrix :")
print(annMatrix)
print()




[2 1 2 2 2 1 0 1 1 2 0 2 2 2 1]

Accuracy : 
0.933333333333

Confusion Matrix :
[[2 0 0]
 [0 4 0]
 [0 1 8]]

[2 2 2 2 2 1 0 1 1 2 0 2 2 2 1]

Accuracy : 
1.0

Confusion Matrix :
[[2 0 0]
 [0 4 0]
 [0 0 9]]



In [30]:
#D 10-fold Cross validation for decision tree and ANN
from sklearn.model_selection import cross_val_score
decisionTree = tree.DecisionTreeClassifier()
multiLayerANN = MLPClassifier(solver = "lbfgs")

treeCrossVal = cross_val_score(decisionTree, X_iris, Y_iris, cv=10)
print("\nTree Score :")
print(treeCrossVal)
print("Accuracy: %0.2f (+/- %0.2f)" % (treeCrossVal.mean(), treeCrossVal.std() * 2))
annCrossVal = cross_val_score(multiLayerANN, X_iris, Y_iris, cv=10)
print("\nANN Score :")
print(annCrossVal)
print("Accuracy: %0.2f (+/- %0.2f)" % (annCrossVal.mean(), annCrossVal.std() * 2))




Tree Score :
[ 1.          0.93333333  1.          0.93333333  0.93333333  0.86666667
  0.93333333  1.          1.          1.        ]
Accuracy: 0.96 (+/- 0.09)

ANN Score :
[ 1.          0.93333333  1.          1.          0.93333333  1.
  0.86666667  1.          1.          1.        ]
Accuracy: 0.97 (+/- 0.09)


In [ ]:
#G Create new instances